In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
from joblib import Parallel, delayed
from download_osm_data import *

### Download latest osm data

In [3]:
#Download data using nominatim geolocator:
download_osm_data('rio', 'south-america-latest.osm.pbf', geo_loc = 'Rio De Janeiro, Brazil')

In [4]:
# Download data using explicit bounding box:
# bbox list -> [lat_start, lat_end, lon_start, lon_end]
download_osm_data('rec', 'south-america-latest.osm.pbf', [-8.155187,-7.928967,-35.018648,-34.851540])

### Donwload historical osm data

In [5]:
download_osm_data('rec_historical', 'south-america-internal.osh.pbf', 
                    geo_loc = 'Recife, Brazil', history = True)

# Here also can use explicit bounding box instead of nominatim geolocator

### Download OSM data for all cities at once

In [3]:
df = pd.read_csv('data/picked_cities.csv') # To download all: more than 110 cities use all_cities.csv

for index, row in df.iterrows():
    city = row['abbr']
    
    bottom_left = eval(row['bottom_left'])
    top_right = eval(row['top_right'])
    osm_file = 'data/osm/latest/'+row['osm_file']+'-latest.osm.pbf'
    bbox = [{'lat': float(bottom_left[0]), 'lon': float(bottom_left[1])}, {'lat': float(top_right[0]), 'lon': float(top_right[1])}] # bottom left, top right

    aux = '%f,%f,%f,%f'%(bbox[0]['lon'],bbox[0]['lat'],bbox[1]['lon'],bbox[1]['lat'])
    filename = 'data/osm/latest/%s.osm.pbf'%city
    
    if not os.path.exists(os.path.dirname(filename)):
        os.makedirs(os.path.dirname(filename))
    
    proc = subprocess.call(['osmium', 'extract', '-b', aux, '-o', filename, '--overwrite', osm_file], shell=True)

### Download OSM history data for all cities at once

In [3]:
df = pd.read_csv('data/picked_cities.csv')

for index, row in df.iterrows():
    city = row['abbr']
    
    bottom_left = eval(row['bottom_left'])
    top_right = eval(row['top_right'])
    osm_file = 'data/osm/historical/'+row['osm_file']+'-internal.osh.pbf'
    bbox = [{'lat': float(bottom_left[0]), 'lon': float(bottom_left[1])}, {'lat': float(top_right[0]), 'lon': float(top_right[1])}] # bottom left, top right

    aux = '%f,%f,%f,%f'%(bbox[0]['lon'],bbox[0]['lat'],bbox[1]['lon'],bbox[1]['lat'])
    filename = 'data/osm/historical/%s.osm.pbf'%city
    
    if not os.path.exists(os.path.dirname(filename)):
        os.makedirs(os.path.dirname(filename))
    
    proc = subprocess.call(['osmium', 'extract', '-b', aux, '-H', '-o', filename, '--overwrite', osm_file], shell=True)